<a href="https://colab.research.google.com/github/WilliamAshbee/splineexample/blob/main/LSTM__wrheadx3_firstpartialsuccessameba.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

With progressive growing of the sequence, the loss drops more stably. 

In [4]:
#TODO:TEST A DIFFERENT TYPE OF WRITE HEAD (ADD A SECOND LTM MEMORY MODULE)
#TODO:TEST A DIFFERENT TYPE OF LOSS TO SMOOTH OUT THE BLURRY READ/WRITE.

https://r2rt.com/non-zero-initial-states-for-recurrent-neural-networks.html

In [5]:
import torch
import numpy as np
import pylab as plt
from skimage import filters
import math

global numpoints
numpoints = 1000
side = 32

rows = torch.zeros(32,32)
columns = torch.zeros(32,32)


for i in range(32):
    columns[:,i] = i
    rows[i,:] = i


def donut_matrix(length = 10):

    radiusMax = side /3
    w = 1
    sigmas = [None, 1]
    
    canvas = torch.zeros((length,side, side))
    r0 = torch.tensor(np.random.uniform(side/4, side/3, length))

    radii = torch.zeros((length,numpoints))
    radii[:, :] = r0.unsqueeze(1)
    
    ind = [x for x in range(numpoints)]

    theta = torch.FloatTensor(ind)
    theta *= math.pi*2.0/(float)(numpoints)
    
    for i in range(1,length):
        a = np.random.uniform(1.0,3.0)*torch.sin(np.random.uniform(20.0)*theta+np.random.uniform(1000.0))
        #a = 4.0*torch.sin(10.0*theta)
        #print(a.shape,torch.max(a))
        radii[i,:] += a
        #print(radii.shape, torch.max(radii))
    
    assert torch.min(radii)>0
    #print(radii.max(axis = 0)[0].shape)
    rmaxs = radii.max(axis = 1)[0]
    pmins = rmaxs+1.0
    pmaxs = side-rmaxs-1.0
    x0 = np.random.uniform(pmins,pmaxs)
    y0 = np.random.uniform(pmins,pmaxs)
    x0[:]=side/2
    y0[:]=side/2
    x0 = torch.tensor(x0)
    y0 = torch.tensor(y0)
    
    x0 = x0.unsqueeze(1)
    y0 = y0.unsqueeze(1)
    #radii = torch.from_numpy(radii)
    xrfactors = torch.cos(theta).unsqueeze(0)
    yrfactors = torch.sin(theta).unsqueeze(0)
    
    print('x0_y0_r_xrf_yrf',x0.shape,y0.shape,radii.shape,xrfactors.shape,yrfactors.shape)

    x = (x0+(xrfactors*radii))
    y = (y0+(yrfactors*radii))
    assert x.shape == (length,numpoints)
    assert y.shape == (length,numpoints)
    assert torch.sum(x[x>(side-1)])==0 
    assert torch.sum(x[x<0])==0 
    assert torch.sum(y[y>(side-1)])==0 
    assert torch.sum(y[y<0])==0 
    
    points = torch.zeros(length,numpoints,2)
    for l in range(length):
        canvas[l,y[l,:].type(torch.LongTensor),x[l,:].type(torch.LongTensor)]=1.0
        points[l,:,0] = x[l,:]#modified for lstm discriminator
        points[l,:,1] = y[l,:]#modified for lstm discriminator 
    
    
    return {
        'canvas': canvas, 
        'points':points.type(torch.FloatTensor)}

def plot_all_model( sample = None, label=None,fig=None,ax=None):
    X = label[:,0]
    Y = label[:,1]
    img = sample[:,:].detach().squeeze().cpu().numpy()
    ax.imshow(img, cmap=plt.cm.gray_r)
    predres = 1000

    s = [.001 for x in range(predres)]
    
    assert len(s) == predres
    c = ['red' for x in range(predres)]
    assert len(c) == predres
    ax.scatter(X.detach().cpu().numpy(),Y.detach().cpu().numpy(),s = s,c = c)


def getXYs(model=None,loader_disp = None):

    outputs = torch.zeros(100,1000,2).cuda()
    h0 = None
    c0 = None
    
    for samples,labels in loader_disp:
      for i in range(38):
        optimizer.zero_grad()
        if i == 0:
          out, h0, c0,o,output = model(samples.cuda(),labels[:,0,:2].cuda(),h0,c0,ind = i)
          outputs[:,i*25:(i+1)*25,:] = out.detach()
        else:
          out, h0, c0,o,output = model(samples.cuda(),labels[:,0,:2].cuda(),h0,c0,ind = i,o = o,output=output)
          outputs[:,i*25:(i+1)*25,:] = out.detach()

      return samples, outputs


          
        

def plot_all( sample = None, labels = None,fig = None,ax = None):
    img = sample[:,:].squeeze().cpu().numpy()
    ax.imshow(img, cmap=plt.cm.gray_r)
    X = labels[:,0]
    Y = labels[:,1]
    s = [.001 for x in range(numpoints)]
    c = ['red' for x in range(numpoints)]
    ax.scatter(X.cpu().numpy(),Y.cpu().numpy(),s = s,c = c)
    
class DonutDataset(torch.utils.data.Dataset):
    """Donut dataset."""
    def __init__(self, length = 10):
        """
        Args:
            csv_file (string): Path to the csv file with annotations.
            root_dir (string): Directory with all the images.
            transform (callable, optional): Optional transform to be applied
                on a sample.
        """
        self.length = length
        self.values = donut_matrix(length)
        assert self.values['canvas'].shape[0] == self.length
        assert self.values['points'].shape[0] == self.length
        
        count = 0
        for i in range(self.length):
          a = self[i]
          c = a[0]
          for el in a[1]:
            #print(c[(int)(el[1]),(int)(el[0])].item())
            #assert c[(int)(el[1]),(int)(el[0])].item() == 1
            y,x = (int)(el[1]),(int)(el[0])
            if x < side-2 and x > 2 and y < side-2 and y > 2: 
              if c[y,x] != 1 and \
                c[y+1,x] != 1 and c[y+1,-1+x] != 1 and c[y+1,1+x] != 1 and \
                c[y-1,x] != 1 and c[y,-1+x] != 1 and c[y,1+x] != 1:
                count+=1
        assert count ==0
    def __len__(self):
        return self.length


    def __getitem__(self, idx):
        canvas = self.values["canvas"]
        
        canvas = canvas[idx,:,:]
        assert canvas.shape == (side,side)
        
        points = self.values["points"]
        points = points[idx,:]
        #points = points.unsqueeze(1)
        #z = torch.zeros(numpoints,1)
        #print(z.shape)
        #points = torch.cat([points,z], dim = 1)
        
        #print('points', points.shape)
        return canvas, points
    
    @staticmethod
    def displayCanvas(title,ddataset, model):
        #model.setBatchSize(batch_size = 1)
        fig = plt.figure()
          
        if model != None:
          #model = model.eval()
          loader_disp = data.DataLoader(
            ddataset, 
            batch_size=100,
            num_workers=2)
          samples, outputs = getXYs(model=model,loader_disp = loader_disp)
          for i in range(100):
            ax = fig.add_subplot(10,10,i+1)
            plot_all_model(samples[i,:,:],outputs[i,:,:],fig=fig,ax=ax)
            plt.axis('off')

        else:
          for i in range(100):
            sample, labels = dataset[i]
            ax = fig.add_subplot(10,10,i+1)
            plot_all(sample = sample,labels = labels,fig = fig, ax= ax)
            plt.axis('off')

        fig.savefig(title,dpi=450)
        plt.close(fig) #where f is the figure
        plt.clf()
        plt.cla()
        plt.close()#should free memory
        print('open file: ',title)


In [6]:
dataset = DonutDataset(length = 100)

DonutDataset.displayCanvas('donut.png',dataset, model = None)


x0_y0_r_xrf_yrf torch.Size([100, 1]) torch.Size([100, 1]) torch.Size([100, 1000]) torch.Size([1, 1000]) torch.Size([1, 1000])
open file:  donut.png


In [7]:

from torch.utils import data
from torch.utils.data import DataLoader, TensorDataset, RandomSampler

mini_batch = 100
dataset = DonutDataset(length = 100*20)
loader_train = data.DataLoader(
    dataset, 
    batch_size=mini_batch,
    sampler=RandomSampler(data_source=dataset),
    num_workers=2)

x0_y0_r_xrf_yrf torch.Size([2000, 1]) torch.Size([2000, 1]) torch.Size([2000, 1000]) torch.Size([1, 1000]) torch.Size([1, 1000])


In [8]:
mini_batch = 100
test_dataset = DonutDataset(length = 1000)
loader_test = data.DataLoader(
    test_dataset, 
    batch_size=mini_batch,
    sampler=RandomSampler(data_source=test_dataset),
    num_workers=2)

x0_y0_r_xrf_yrf torch.Size([1000, 1]) torch.Size([1000, 1]) torch.Size([1000, 1000]) torch.Size([1, 1000]) torch.Size([1, 1000])


In [9]:
pred_dataset = DonutDataset(length = 100)

x0_y0_r_xrf_yrf torch.Size([100, 1]) torch.Size([100, 1]) torch.Size([100, 1000]) torch.Size([1, 1000]) torch.Size([1, 1000])


In [10]:
import torch
import torch.nn as nn
from torch.hub import load_state_dict_from_url

class DecoderRNN(nn.Module):
    def __init__(self):
        super(DecoderRNN, self).__init__()
        self.track = 0
        # define the properties
        self.embed_size = 2
        self.hidden_size = 256
        self.num_layers = 2
        self.seq_len = 25
        self.full_len = 1000
        self.longmem_len = int(self.full_len/self.seq_len)
        self.longtermMem = None
        self.longtermMemP = None
        self.lstm_cell = nn.LSTM(self.hidden_size*2, self.hidden_size,self.num_layers)

        self.inject_image =  nn.Sequential(
            nn.Linear(2*1000+1024+6*self.hidden_size,self.hidden_size*2),
            nn.ReLU()
        )

        
        self.fc_out = nn.Sequential(
            nn.Linear(2+self.hidden_size*8,10000),
            nn.ReLU(),
            nn.Dropout(),
            nn.Linear(10000,2),
            nn.Sigmoid()
        )
        self.hi_1 = nn.Sequential(
            nn.Linear(2+1024+self.hidden_size*2,self.hidden_size),
        )
        self.hi_2 = nn.Sequential(
            nn.Linear(2+1024+self.hidden_size*2,self.hidden_size),
        )

        self.ci_1 = nn.Sequential(
            nn.Linear(2+1024+self.hidden_size*2,self.hidden_size),
        )
        self.ci_2 = nn.Sequential(
            nn.Linear(2+1024+self.hidden_size*2,self.hidden_size),
        )

        self.alpha_1 = nn.Sequential(
            nn.Linear(self.hidden_size*2,self.seq_len),
            nn.Softmax()
        )
        
        self.alpha_long1 = nn.Sequential(
            nn.Linear(self.hidden_size*2,self.longmem_len),
            nn.Softmax()
        )
        
        self.alpha_long2 = nn.Sequential(
            nn.Linear(self.hidden_size*2,self.longmem_len),
            nn.Softmax()
        )
        
        self.alpha_long3 = nn.Sequential(
            nn.Linear(self.hidden_size*2,self.longmem_len),
            nn.Softmax()
        )
        
        self.long_term_write = nn.Sequential(
            nn.Linear(self.hidden_size*(self.seq_len+1),self.hidden_size),
            nn.ReLU()
        )
        

        self.embs = None #= torch.empty((self.batch_size, 1000, self.hidden_size)).cuda()
        self.ltm_temp = None
        

            
        
    def forward(self, features, p0,h0 = None, c0= None, ind = 0, o = None,output = None):
        #print('a')
        assert features != None
        self.track +=1
        batch_size = features.size(0)
        

        
        
        if o == None:
          out = p0
          self.longtermMem = torch.zeros(batch_size,self.longmem_len,self.hidden_size).cuda()
          self.embs = torch.empty((batch_size, self.seq_len, self.hidden_size)).cuda()
          self.longtermMemP = torch.zeros(batch_size,1000,2).cuda()
          self.ltm_temp = torch.zeros(batch_size,self.hidden_size).cuda()
        else:

          self.ltm_temp = self.long_term_write(torch.cat([torch.flatten(self.embs,start_dim=1),output.squeeze()],dim=1))
          self.longtermMem[:, ind-1, :] = self.ltm_temp
        
          self.embs = torch.empty((batch_size, self.seq_len, self.hidden_size)).cuda()
          
          out = o

        #print('c')
        if h0 == None and c0 == None:
          combin_init = torch.cat([torch.flatten(features,start_dim=1),out,torch.zeros(batch_size,self.hidden_size).cuda(),torch.zeros(batch_size,self.hidden_size).cuda()],dim=1)
          h_0_1 = self.hi_1(combin_init)  
          h_0_2 = self.hi_2(combin_init)  
          c_0_1 = self.ci_1(combin_init)  
          c_0_2 = self.ci_2(combin_init)  
          
          h0 = torch.stack([h_0_1,h_0_2],dim = 0)
          c0 = torch.stack([c_0_1,c_0_2],dim = 0)
        else:
          combin_init = torch.cat([torch.flatten(features,start_dim=1),out,output.squeeze(),h0[0,:,:]],dim=1)
          h_0_1 = self.hi_1(combin_init)  
          
          combin_init = torch.cat([torch.flatten(features,start_dim=1),out,output.squeeze(),h0[1,:,:]],dim=1)
          h_0_2 = self.hi_2(combin_init)  
          
          combin_init = torch.cat([torch.flatten(features,start_dim=1),out,output.squeeze(),c0[0,:,:]],dim=1)
          c_0_1 = self.ci_1(combin_init)  
          
          combin_init = torch.cat([torch.flatten(features,start_dim=1),out,output.squeeze(),c0[1,:,:]],dim=1)
          c_0_2 = self.ci_2(combin_init)
          
          h0 = torch.stack([h_0_1,h_0_2],dim = 0)
          c0 = torch.stack([c_0_1,c_0_2],dim = 0)
        
        
        hidden_state = h0
        cell_state = c0
        outputs = torch.empty((batch_size, self.seq_len, 2)).cuda()
        
        output = None
        #print('b')
        for t in range(self.seq_len):
        
            # for the first time step the input is the feature vector
            if output == None:
              output = torch.zeros(batch_size,self.hidden_size).cuda()
              outesread1 = torch.zeros(batch_size,self.hidden_size).cuda()
              outesreadl1 = torch.zeros(batch_size,self.hidden_size).cuda()
              outesreadl2 = torch.zeros(batch_size,self.hidden_size).cuda()
              outesreadl3 = torch.zeros(batch_size,self.hidden_size).cuda()
            
            combin_lstm = torch.cat([torch.flatten(features,start_dim=1),torch.flatten(self.longtermMemP,start_dim=1),output.squeeze(),outesread1,outesreadl1,outesreadl2,outesreadl3,self.ltm_temp],dim=1)
            combin_lstm = self.inject_image(combin_lstm)
            
            a1 = self.alpha_1(combin_lstm)
            outesreada1 = self.embs.clone()*a1.unsqueeze(2)#clone
            outesread1 = torch.sum(outesreada1,dim=1)
            
            
            
            output, (hidden_state, cell_state) = self.lstm_cell(combin_lstm.unsqueeze(0), (hidden_state, cell_state))

            outputs[:, t, :] = out
            self.longtermMemP[:,self.seq_len*ind+t,:] = out.detach()
            
            self.embs[:, t, :] = output.squeeze().clone().detach()
            

            a1 = self.alpha_1(combin_lstm)
            outesreada1 = self.embs.clone()*a1.unsqueeze(2)#clone
            outesread1 = torch.sum(outesreada1,dim=1)
            
            
            #print('d')
            al1 = self.alpha_long1(combin_lstm)
            outesreadal1 = self.longtermMem.detach()*al1.unsqueeze(2)#clone
            outesreadl1 = torch.sum(outesreadal1,dim=1)
            
            al2 = self.alpha_long2(combin_lstm)
            outesreadal2 = self.longtermMem.detach()*al2.unsqueeze(2)#clone
            outesreadl2 = torch.sum(outesreadal2,dim=1)
            
            al3 = self.alpha_long3(combin_lstm)
            outesreadal3 = self.longtermMem.detach()*al3.unsqueeze(2)#clone
            outesreadl3 = torch.sum(outesreadal3,dim=1)
            

            combOut = torch.cat([combin_lstm, output.squeeze(),out,outesread1,outesreadl1,outesreadl2,outesreadl3,self.ltm_temp],dim=1)#,outesread],dim=1)
            

            out = self.fc_out(combOut)
            out = out*32.0
            # build the output tensor
            
        
            
        #self.embs = self.embs.detach().clone()
        return outputs, hidden_state.detach(), cell_state.detach(), out.detach(),output.detach()#, captions


In [11]:
model = DecoderRNN().cuda()

In [12]:
optimizer = torch.optim.Adam(model.parameters(),lr = 0.0001, betas = (.9,.999))#ideal


In [13]:
import torch
from torch.utils.tensorboard import SummaryWriter
writer = SummaryWriter()


In [14]:
for seql in range(3,23,5):
  #seql = 19
  for epoch in range (5):
    loss = None
    #model = model.train()

    for x,y in loader_train:
      h0 = None
      c0 = None
      for i in range(seql):
        x = x.cuda()
        y = y.cuda()
        
        optimizer.zero_grad()
        
        if i == 0:
          out, h0, c0,o,output = model(x,y[:,0,:2],h0,c0,ind = i)
        else:
          out, h0, c0,o,output = model(x,y[:,0,:2],h0,c0,ind = i,o = o,output=output)
        loss = torch.mean((out-y[:,1+(25*i):1+(25*(i+1)),:2])**2)
        #writer.add_scalar("Loss/train", loss, epoch)
        
        loss.backward()
        optimizer.step()
      #print(y[:,0,:2])
      print('epoch',epoch,'seql',seql)
      print('train loss', loss)
    
    with torch.no_grad():
      #model = model.eval()
      for x,y in loader_test:
        h0 = None
        c0 = None
        for i in range(seql):
          x = x.cuda()
          y = y.cuda()
          
          if i == 0:
            out, h0, c0,o,output = model(x,y[:,0,:2],h0,c0,ind = i)
          else:
            out, h0, c0,o,output = model(x,y[:,0,:2],h0,c0,ind = i,o = o,output=output)
          
          loss = torch.mean((out-y[:,1+(25*i):1+(25*(i+1)),:2])**2)
        
          assert not torch.any(torch.isnan(out)).item()
          assert not torch.any(torch.isnan(y)).item()
        
        print('epoch',epoch,'seql',seql)
        print('testloss',loss)
  
        

/usr/local/lib/python3.7/dist-packages/torch/nn/modules/container.py:119: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  input = module(input)


epoch 0 seql 3
train loss tensor(11.2958, device='cuda:0', grad_fn=<MeanBackward0>)
epoch 0 seql 3
train loss tensor(19.7675, device='cuda:0', grad_fn=<MeanBackward0>)
epoch 0 seql 3
train loss tensor(4.4841, device='cuda:0', grad_fn=<MeanBackward0>)
epoch 0 seql 3
train loss tensor(20.3010, device='cuda:0', grad_fn=<MeanBackward0>)
epoch 0 seql 3
train loss tensor(7.9217, device='cuda:0', grad_fn=<MeanBackward0>)
epoch 0 seql 3
train loss tensor(3.3267, device='cuda:0', grad_fn=<MeanBackward0>)
epoch 0 seql 3
train loss tensor(4.2109, device='cuda:0', grad_fn=<MeanBackward0>)
epoch 0 seql 3
train loss tensor(3.3858, device='cuda:0', grad_fn=<MeanBackward0>)
epoch 0 seql 3
train loss tensor(3.2804, device='cuda:0', grad_fn=<MeanBackward0>)
epoch 0 seql 3
train loss tensor(3.1988, device='cuda:0', grad_fn=<MeanBackward0>)
epoch 0 seql 3
train loss tensor(2.9996, device='cuda:0', grad_fn=<MeanBackward0>)
epoch 0 seql 3
train loss tensor(3.2657, device='cuda:0', grad_fn=<MeanBackward0>)
e

In [15]:
with torch.no_grad():
  #model = model.eval()
  DonutDataset.displayCanvas('intermediate1.png',pred_dataset, model = model)

/usr/local/lib/python3.7/dist-packages/torch/nn/modules/container.py:119: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  input = module(input)


open file:  intermediate1.png


In [16]:
#optimizer = torch.optim.Adam(model.parameters(),lr = 0.0001, betas = (.9,.999))#ideal

for g in optimizer.param_groups:
    g['lr'] = 0.0001


In [17]:
#for seql in range(15,20,4):
seql = 23
for epoch in range (20):
  #model = model.train()
  loss = None
  for x,y in loader_train:
    h0 = None
    c0 = None
    for i in range(seql):
      x = x.cuda()
      y = y.cuda()
      
      optimizer.zero_grad()
      
      if i == 0:
        out, h0, c0,o,output = model(x,y[:,0,:2],h0,c0,ind = i)
      else:
        out, h0, c0,o,output = model(x,y[:,0,:2],h0,c0,ind = i,o = o,output=output)
      loss = torch.mean((out-y[:,1+(25*i):1+(25*(i+1)),:2])**2)
      writer.add_scalar("Loss/train", loss, epoch)
      
      loss.backward()
      optimizer.step()
    #print(y[:,0,:2])
    print('epoch',epoch,'seql',seql)
    print(loss)
  with torch.no_grad():
    #model = model.eval()
    for x,y in loader_test:
      h0 = None
      c0 = None
      for i in range(seql):
        x = x.cuda()
        y = y.cuda()
        
        if i == 0:
          out, h0, c0,o,output = model(x,y[:,0,:2],h0,c0,ind = i)
        else:
          out, h0, c0,o,output = model(x,y[:,0,:2],h0,c0,ind = i,o = o,output=output)
        
        loss = torch.mean((out-y[:,1+(25*i):1+(25*(i+1)),:2])**2)
      
        assert not torch.any(torch.isnan(out)).item()
        assert not torch.any(torch.isnan(y)).item()
      
      print('epoch',epoch,'seql',seql)
      print('testloss',loss)


/usr/local/lib/python3.7/dist-packages/torch/nn/modules/container.py:119: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  input = module(input)


epoch 0 seql 23
tensor(7.0259, device='cuda:0', grad_fn=<MeanBackward0>)
epoch 0 seql 23
tensor(13.1362, device='cuda:0', grad_fn=<MeanBackward0>)
epoch 0 seql 23
tensor(14.3912, device='cuda:0', grad_fn=<MeanBackward0>)
epoch 0 seql 23
tensor(2.7504, device='cuda:0', grad_fn=<MeanBackward0>)
epoch 0 seql 23
tensor(7.0421, device='cuda:0', grad_fn=<MeanBackward0>)
epoch 0 seql 23
tensor(1.4843, device='cuda:0', grad_fn=<MeanBackward0>)
epoch 0 seql 23
tensor(1.7845, device='cuda:0', grad_fn=<MeanBackward0>)
epoch 0 seql 23
tensor(8.3602, device='cuda:0', grad_fn=<MeanBackward0>)
epoch 0 seql 23
tensor(3.1905, device='cuda:0', grad_fn=<MeanBackward0>)
epoch 0 seql 23
tensor(1.5871, device='cuda:0', grad_fn=<MeanBackward0>)
epoch 0 seql 23
tensor(1.8902, device='cuda:0', grad_fn=<MeanBackward0>)
epoch 0 seql 23
tensor(15.1772, device='cuda:0', grad_fn=<MeanBackward0>)
epoch 0 seql 23
tensor(2.5051, device='cuda:0', grad_fn=<MeanBackward0>)
epoch 0 seql 23
tensor(1.9523, device='cuda:0', 

In [18]:
with torch.no_grad():
  #model = model.eval()
  DonutDataset.displayCanvas('intermediate2.png',pred_dataset, model = model)

/usr/local/lib/python3.7/dist-packages/torch/nn/modules/container.py:119: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  input = module(input)


open file:  intermediate2.png


In [19]:
#optimizer = torch.optim.Adam(model.parameters(),lr = 0.00001, betas = (.9,.999))#ideal
for g in optimizer.param_groups:
    g['lr'] = 0.00001



In [20]:

#model = model.train()
#for seql in range(15,20,4):
seql = 23
for epoch in range (10):
  loss = None
  #model = model.train()
  for x,y in loader_train:
    h0 = None
    c0 = None
    for i in range(seql):
      x = x.cuda()
      y = y.cuda()
      
      optimizer.zero_grad()
      
      if i == 0:
        out, h0, c0,o,output = model(x,y[:,0,:2],h0,c0,ind = i)
      else:
        out, h0, c0,o,output = model(x,y[:,0,:2],h0,c0,ind = i,o = o,output=output)
      loss = torch.mean((out-y[:,1+(25*i):1+(25*(i+1)),:2])**2)
      writer.add_scalar("Loss/train", loss, epoch)
      
      loss.backward()
      optimizer.step()
    #print(y[:,0,:2])
    print('epoch',epoch,'seql',seql)
    print(loss)
  with torch.no_grad():
      #model = model.eval()
      for x,y in loader_test:
        h0 = None
        c0 = None
        for i in range(seql):
          x = x.cuda()
          y = y.cuda()
          
          if i == 0:
            out, h0, c0,o,output = model(x,y[:,0,:2],h0,c0,ind = i)
          else:
            out, h0, c0,o,output = model(x,y[:,0,:2],h0,c0,ind = i,o = o,output=output)
          
          loss = torch.mean((out-y[:,1+(25*i):1+(25*(i+1)),:2])**2)
        
          assert not torch.any(torch.isnan(out)).item()
          assert not torch.any(torch.isnan(y)).item()
        
        print('epoch',epoch,'seql',seql)
        print('testloss',loss)
  
        

/usr/local/lib/python3.7/dist-packages/torch/nn/modules/container.py:119: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  input = module(input)


epoch 0 seql 23
tensor(0.8967, device='cuda:0', grad_fn=<MeanBackward0>)
epoch 0 seql 23
tensor(0.7248, device='cuda:0', grad_fn=<MeanBackward0>)
epoch 0 seql 23
tensor(0.8045, device='cuda:0', grad_fn=<MeanBackward0>)
epoch 0 seql 23
tensor(0.6587, device='cuda:0', grad_fn=<MeanBackward0>)
epoch 0 seql 23
tensor(0.6741, device='cuda:0', grad_fn=<MeanBackward0>)
epoch 0 seql 23
tensor(0.6716, device='cuda:0', grad_fn=<MeanBackward0>)
epoch 0 seql 23
tensor(0.6682, device='cuda:0', grad_fn=<MeanBackward0>)
epoch 0 seql 23
tensor(0.6438, device='cuda:0', grad_fn=<MeanBackward0>)
epoch 0 seql 23
tensor(0.6111, device='cuda:0', grad_fn=<MeanBackward0>)
epoch 0 seql 23
tensor(0.5930, device='cuda:0', grad_fn=<MeanBackward0>)
epoch 0 seql 23
tensor(0.6766, device='cuda:0', grad_fn=<MeanBackward0>)
epoch 0 seql 23
tensor(0.5887, device='cuda:0', grad_fn=<MeanBackward0>)
epoch 0 seql 23
tensor(0.6199, device='cuda:0', grad_fn=<MeanBackward0>)
epoch 0 seql 23
tensor(0.5576, device='cuda:0', gra

ran so well, interrupted

In [21]:
with torch.no_grad():
  #model = model.eval()
  DonutDataset.displayCanvas('intermediate3.png',pred_dataset, model = model)

/usr/local/lib/python3.7/dist-packages/torch/nn/modules/container.py:119: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  input = module(input)


open file:  intermediate3.png


In [22]:
#optimizer = torch.optim.Adam(model.parameters(),lr = 0.00001, betas = (.9,.999))#ideal
for g in optimizer.param_groups:
    g['lr'] = 0.0001



In [23]:
for seql in range(23,39,4):
  #seql = 19
  for epoch in range (5):
    loss = None
    #model = model.train()

    for x,y in loader_train:
      h0 = None
      c0 = None
      for i in range(seql):
        x = x.cuda()
        y = y.cuda()
        
        optimizer.zero_grad()
        
        if i == 0:
          out, h0, c0,o,output = model(x,y[:,0,:2],h0,c0,ind = i)
        else:
          out, h0, c0,o,output = model(x,y[:,0,:2],h0,c0,ind = i,o = o,output=output)
        loss = torch.mean((out-y[:,1+(25*i):1+(25*(i+1)),:2])**2)
        #writer.add_scalar("Loss/train", loss, epoch)
        
        loss.backward()
        optimizer.step()
      #print(y[:,0,:2])
      print('epoch',epoch,'seql',seql)
      print('train loss', loss)
    
    with torch.no_grad():
      #model = model.eval()
      for x,y in loader_test:
        h0 = None
        c0 = None
        for i in range(seql):
          x = x.cuda()
          y = y.cuda()
          
          if i == 0:
            out, h0, c0,o,output = model(x,y[:,0,:2],h0,c0,ind = i)
          else:
            out, h0, c0,o,output = model(x,y[:,0,:2],h0,c0,ind = i,o = o,output=output)
          
          loss = torch.mean((out-y[:,1+(25*i):1+(25*(i+1)),:2])**2)
        
          assert not torch.any(torch.isnan(out)).item()
          assert not torch.any(torch.isnan(y)).item()
        
        print('epoch',epoch,'seql',seql)
        print('testloss',loss)
  
        

/usr/local/lib/python3.7/dist-packages/torch/nn/modules/container.py:119: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  input = module(input)


epoch 0 seql 23
train loss tensor(1.2098, device='cuda:0', grad_fn=<MeanBackward0>)
epoch 0 seql 23
train loss tensor(1.6663, device='cuda:0', grad_fn=<MeanBackward0>)
epoch 0 seql 23
train loss tensor(4.0121, device='cuda:0', grad_fn=<MeanBackward0>)
epoch 0 seql 23
train loss tensor(5.8787, device='cuda:0', grad_fn=<MeanBackward0>)
epoch 0 seql 23
train loss tensor(6.2058, device='cuda:0', grad_fn=<MeanBackward0>)
epoch 0 seql 23
train loss tensor(3.4334, device='cuda:0', grad_fn=<MeanBackward0>)
epoch 0 seql 23
train loss tensor(2.1804, device='cuda:0', grad_fn=<MeanBackward0>)
epoch 0 seql 23
train loss tensor(3.5567, device='cuda:0', grad_fn=<MeanBackward0>)
epoch 0 seql 23
train loss tensor(12.2264, device='cuda:0', grad_fn=<MeanBackward0>)
epoch 0 seql 23
train loss tensor(3.2563, device='cuda:0', grad_fn=<MeanBackward0>)
epoch 0 seql 23
train loss tensor(9.3596, device='cuda:0', grad_fn=<MeanBackward0>)
epoch 0 seql 23
train loss tensor(3.0349, device='cuda:0', grad_fn=<MeanBac

In [24]:
with torch.no_grad():
  #model = model.eval()
  DonutDataset.displayCanvas('intermediate4.png',pred_dataset, model = model)

/usr/local/lib/python3.7/dist-packages/torch/nn/modules/container.py:119: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  input = module(input)


open file:  intermediate4.png


In [25]:
#optimizer = torch.optim.Adam(model.parameters(),lr = 0.00001, betas = (.9,.999))#ideal
for g in optimizer.param_groups:
    g['lr'] = 0.0001



In [26]:
#for seql in range(15,20,4):
seql = 38
for epoch in range (20):
  #model = model.train()
  loss = None
  for x,y in loader_train:
    h0 = None
    c0 = None
    for i in range(seql):
      x = x.cuda()
      y = y.cuda()
      
      optimizer.zero_grad()
      
      if i == 0:
        out, h0, c0,o,output = model(x,y[:,0,:2],h0,c0,ind = i)
      else:
        out, h0, c0,o,output = model(x,y[:,0,:2],h0,c0,ind = i,o = o,output=output)
      loss = torch.mean((out-y[:,1+(25*i):1+(25*(i+1)),:2])**2)
      writer.add_scalar("Loss/train", loss, epoch)
      
      loss.backward()
      optimizer.step()
    #print(y[:,0,:2])
    print('epoch',epoch,'seql',seql)
    print(loss)
  with torch.no_grad():
    #model = model.eval()
    for x,y in loader_test:
      h0 = None
      c0 = None
      for i in range(seql):
        x = x.cuda()
        y = y.cuda()
        
        if i == 0:
          out, h0, c0,o,output = model(x,y[:,0,:2],h0,c0,ind = i)
        else:
          out, h0, c0,o,output = model(x,y[:,0,:2],h0,c0,ind = i,o = o,output=output)
        
        loss = torch.mean((out-y[:,1+(25*i):1+(25*(i+1)),:2])**2)
      
        assert not torch.any(torch.isnan(out)).item()
        assert not torch.any(torch.isnan(y)).item()
      
      print('epoch',epoch,'seql',seql)
      print('testloss',loss)


/usr/local/lib/python3.7/dist-packages/torch/nn/modules/container.py:119: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  input = module(input)


epoch 0 seql 38
tensor(14.3647, device='cuda:0', grad_fn=<MeanBackward0>)
epoch 0 seql 38
tensor(16.1279, device='cuda:0', grad_fn=<MeanBackward0>)
epoch 0 seql 38
tensor(16.0898, device='cuda:0', grad_fn=<MeanBackward0>)
epoch 0 seql 38
tensor(14.0836, device='cuda:0', grad_fn=<MeanBackward0>)
epoch 0 seql 38
tensor(24.4231, device='cuda:0', grad_fn=<MeanBackward0>)
epoch 0 seql 38
tensor(13.9033, device='cuda:0', grad_fn=<MeanBackward0>)
epoch 0 seql 38
tensor(11.4683, device='cuda:0', grad_fn=<MeanBackward0>)
epoch 0 seql 38
tensor(14.2151, device='cuda:0', grad_fn=<MeanBackward0>)
epoch 0 seql 38
tensor(14.2948, device='cuda:0', grad_fn=<MeanBackward0>)
epoch 0 seql 38
tensor(23.8403, device='cuda:0', grad_fn=<MeanBackward0>)
epoch 0 seql 38
tensor(21.4774, device='cuda:0', grad_fn=<MeanBackward0>)
epoch 0 seql 38
tensor(17.8780, device='cuda:0', grad_fn=<MeanBackward0>)
epoch 0 seql 38
tensor(12.6284, device='cuda:0', grad_fn=<MeanBackward0>)
epoch 0 seql 38
tensor(13.4278, device

In [27]:
with torch.no_grad():
  #model = model.eval()
  DonutDataset.displayCanvas('intermediate5.png',pred_dataset, model = model)

/usr/local/lib/python3.7/dist-packages/torch/nn/modules/container.py:119: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  input = module(input)


open file:  intermediate5.png


In [28]:
#optimizer = torch.optim.Adam(model.parameters(),lr = 0.00001, betas = (.9,.999))#ideal
for g in optimizer.param_groups:
    g['lr'] = 0.00001



In [29]:

#model = model.train()
#for seql in range(15,20,4):
seql = 38
for epoch in range (60):
  loss = None
  #model = model.train()
  for x,y in loader_train:
    h0 = None
    c0 = None
    for i in range(seql):
      x = x.cuda()
      y = y.cuda()
      
      optimizer.zero_grad()
      
      if i == 0:
        out, h0, c0,o,output = model(x,y[:,0,:2],h0,c0,ind = i)
      else:
        out, h0, c0,o,output = model(x,y[:,0,:2],h0,c0,ind = i,o = o,output=output)
      loss = torch.mean((out-y[:,1+(25*i):1+(25*(i+1)),:2])**2)
      writer.add_scalar("Loss/train", loss, epoch)
      
      loss.backward()
      optimizer.step()
    #print(y[:,0,:2])
    print('epoch',epoch,'seql',seql)
    print(loss)
  with torch.no_grad():
      #model = model.eval()
      for x,y in loader_test:
        h0 = None
        c0 = None
        for i in range(seql):
          x = x.cuda()
          y = y.cuda()
          
          if i == 0:
            out, h0, c0,o,output = model(x,y[:,0,:2],h0,c0,ind = i)
          else:
            out, h0, c0,o,output = model(x,y[:,0,:2],h0,c0,ind = i,o = o,output=output)
          
          loss = torch.mean((out-y[:,1+(25*i):1+(25*(i+1)),:2])**2)
        
          assert not torch.any(torch.isnan(out)).item()
          assert not torch.any(torch.isnan(y)).item()
        
        print('epoch',epoch,'seql',seql)
        print('testloss',loss)
  
        

/usr/local/lib/python3.7/dist-packages/torch/nn/modules/container.py:119: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  input = module(input)


epoch 0 seql 38
tensor(1.8106, device='cuda:0', grad_fn=<MeanBackward0>)
epoch 0 seql 38
tensor(2.7015, device='cuda:0', grad_fn=<MeanBackward0>)
epoch 0 seql 38
tensor(2.5854, device='cuda:0', grad_fn=<MeanBackward0>)
epoch 0 seql 38
tensor(2.4189, device='cuda:0', grad_fn=<MeanBackward0>)
epoch 0 seql 38
tensor(2.5354, device='cuda:0', grad_fn=<MeanBackward0>)
epoch 0 seql 38
tensor(2.5100, device='cuda:0', grad_fn=<MeanBackward0>)
epoch 0 seql 38
tensor(2.2931, device='cuda:0', grad_fn=<MeanBackward0>)
epoch 0 seql 38
tensor(2.2538, device='cuda:0', grad_fn=<MeanBackward0>)
epoch 0 seql 38
tensor(2.1178, device='cuda:0', grad_fn=<MeanBackward0>)
epoch 0 seql 38
tensor(2.1212, device='cuda:0', grad_fn=<MeanBackward0>)
epoch 0 seql 38
tensor(2.2469, device='cuda:0', grad_fn=<MeanBackward0>)
epoch 0 seql 38
tensor(2.4332, device='cuda:0', grad_fn=<MeanBackward0>)
epoch 0 seql 38
tensor(1.9196, device='cuda:0', grad_fn=<MeanBackward0>)
epoch 0 seql 38
tensor(1.9644, device='cuda:0', gra

In [30]:
writer.flush()
writer.close()


In [31]:
#%load_ext tensorboard

In [32]:
#!kill 3285
#%tensorboard --logdir runs

In [33]:
#!nvidia-smi

In [34]:
with torch.no_grad():
  #model = model.eval()
  DonutDataset.displayCanvas('lstmpredictions.png',pred_dataset, model = model)

/usr/local/lib/python3.7/dist-packages/torch/nn/modules/container.py:119: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  input = module(input)


open file:  lstmpredictions.png


In [35]:
for name, param in model.named_parameters():
  print(name)

lstm_cell.weight_ih_l0
lstm_cell.weight_hh_l0
lstm_cell.bias_ih_l0
lstm_cell.bias_hh_l0
lstm_cell.weight_ih_l1
lstm_cell.weight_hh_l1
lstm_cell.bias_ih_l1
lstm_cell.bias_hh_l1
inject_image.0.weight
inject_image.0.bias
fc_out.0.weight
fc_out.0.bias
fc_out.3.weight
fc_out.3.bias
hi_1.0.weight
hi_1.0.bias
hi_2.0.weight
hi_2.0.bias
ci_1.0.weight
ci_1.0.bias
ci_2.0.weight
ci_2.0.bias
alpha_1.0.weight
alpha_1.0.bias
alpha_long1.0.weight
alpha_long1.0.bias
alpha_long2.0.weight
alpha_long2.0.bias
alpha_long3.0.weight
alpha_long3.0.bias
long_term_write.0.weight
long_term_write.0.bias


In [36]:
with torch.no_grad():
      #model = model.eval()
      for x,y in loader_test:
        h0 = None
        c0 = None
        for i in range(seql):
          x = x.cuda()
          y = y.cuda()
          
          if i == 0:
            out, h0, c0,o,output = model(x,y[:,0,:2],h0,c0,ind = i)
          else:
            out, h0, c0,o,output = model(x,y[:,0,:2],h0,c0,ind = i,o = o,output=output)
          
          loss = torch.mean((out-y[:,1+(25*i):1+(25*(i+1)),:2])**2)
        
          assert not torch.any(torch.isnan(out)).item()
          assert not torch.any(torch.isnan(y)).item()
        
        print('epoch',epoch,'seql',seql)
        print('testloss',loss)
  
        

/usr/local/lib/python3.7/dist-packages/torch/nn/modules/container.py:119: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  input = module(input)


epoch 59 seql 38
testloss tensor(1.2771, device='cuda:0')
epoch 59 seql 38
testloss tensor(1.1749, device='cuda:0')
epoch 59 seql 38
testloss tensor(1.4375, device='cuda:0')
epoch 59 seql 38
testloss tensor(1.4069, device='cuda:0')
epoch 59 seql 38
testloss tensor(1.3851, device='cuda:0')
epoch 59 seql 38
testloss tensor(1.4252, device='cuda:0')
epoch 59 seql 38
testloss tensor(1.1831, device='cuda:0')
epoch 59 seql 38
testloss tensor(1.5342, device='cuda:0')
epoch 59 seql 38
testloss tensor(1.2177, device='cuda:0')
epoch 59 seql 38
testloss tensor(1.3115, device='cuda:0')
